In [30]:
#!/usr/bin/env python3
# create_individual_files.py

import os
import shutil
import pandas as pd
import math
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment
from openpyxl.utils import get_column_letter

# === Einstellungen ===
FONT = Font(name='Arial', size=8)
FONT_BOLD = Font(name='Arial', size=8, bold=True)
thin = Side(border_style='thin', color='000000')
border = Border(left=thin, right=thin, top=thin, bottom=thin)

# Hilfsfunktion: Formatiert Zahl mit sig signifikanten Ziffern, Komma als Dezimaltrennzeichen
def format_sig(x, sig=4):
    if pd.isna(x):
        return ''
    if x == 0:
        return '0'
    order = int(math.floor(math.log10(abs(x))))
    decimals = sig - 1 - order
    if decimals < 0:
        decimals = 0
    s = f"{x:.{decimals}f}"
    return s.replace('.', ',')

# Sortierreihenfolge für Modelle
MODEL_ORDER = ['Pi4b', 'Pi3b', 'PiZero2', 'PiBanana', 'PiZero', 'Pi1bPlus', 'Pi1b']

# Pfade
df_path = 'output/csv/nist_all.csv'
out_dir  = 'output/xlsx/nist_scaled'

# Output-Verzeichnis neu anlegen
if os.path.exists(out_dir):
    shutil.rmtree(out_dir)
os.makedirs(out_dir, exist_ok=True)

# CSV einlesen
raw = pd.read_csv(df_path)
raw.columns = raw.columns.str.lower()

# Pro Algorithmus verarbeiten
for algo, grp in raw.groupby('algorithmus'):
    # Picnic-Verfahren überspringen
    if algo.lower().startswith('picnic'):
        print(f"Überspringe Verfahren {algo}")
        continue
        
    safe = algo.replace('/', '_').replace(' ', '_')
    wb   = Workbook()
    ws   = wb.active
    ws.title = 'NIST All'

    # === Model-Überschrift: Merge A1:A2 und vollständiger Rahmen ===
    ws.merge_cells(start_row=1, start_column=1, end_row=2, end_column=1)
    for r in (1, 2):
        model_cell = ws.cell(row=r, column=1)
        if r == 1:
            model_cell.value = 'Model'
            model_cell.font  = FONT_BOLD
        # Alignment auch für die zweite Zelle
        model_cell.alignment = Alignment(horizontal='center', vertical='center')
        # Rahmen auf beide Zellen anwenden
        model_cell.border = border

    # Typ (kem/sig)
    typ = 'kem'
    if 'typ' in grp.columns and grp['typ'].notna().any():
        typ = str(grp['typ'].dropna().iloc[0]).lower()
    sub_map = {'Enc': 'Sign', 'Dec': 'Verify'} if typ == 'sig' else {'Enc': 'Enc', 'Dec': 'Dec'}

    # Roh-Daten
    df = pd.DataFrame({
        'model':        grp['model'],
        'keygen_time':  grp['keygen_time'],
        'enc_time':     grp['enc_time'],
        'dec_time':     grp['dec_time'],
        'keygen_heap':  grp['keygen_heap'],
        'enc_heap':     grp['enc_heap'],
        'dec_heap':     grp['dec_heap'],
        'keygen_stack': grp['keygen_stack'],
        'enc_stack':    grp['enc_stack'],
        'dec_stack':    grp['dec_stack'],
    })
    df['model'] = pd.Categorical(df['model'], categories=MODEL_ORDER, ordered=True)
    df = df.sort_values('model')

    # Dynamische Skalierung basierend auf kleinstem Wert ≥ Einheit
    # Time (µs, ms, s)
    times = df[['keygen_time','enc_time','dec_time']].values.flatten()
    nz_t = times[times > 0]
    min_t = nz_t.min() if len(nz_t) else 0
    if min_t >= 1e6:
        t_scale, t_unit = 1e6, 's'
    elif min_t >= 1e3:
        t_scale, t_unit = 1e3, 'ms'
    else:
        t_scale, t_unit = 1, 'µs'
    # Heap (B, KiB, MiB)
    heaps = df[['keygen_heap','enc_heap','dec_heap']].values.flatten()
    nz_h = heaps[heaps > 0]
    min_h = nz_h.min() if len(nz_h) else 0
    if min_h >= 2**20:
        h_scale, h_unit = 2**20, 'MiB'
    elif min_h >= 2**10:
        h_scale, h_unit = 2**10, 'KiB'
    else:
        h_scale, h_unit = 1, 'B'
    # Stack (B, KiB, MiB)
    stacks = df[['keygen_stack','enc_stack','dec_stack']].values.flatten()
    nz_s = stacks[stacks > 0]
    min_s = nz_s.min() if len(nz_s) else 0
    if min_s >= 2**20:
        s_scale, s_unit = 2**20, 'MiB'
    elif min_s >= 2**10:
        s_scale, s_unit = 2**10, 'KiB'
    else:
        s_scale, s_unit = 1, 'B'

    # Skalierung und Formatierung als Text
    if t_scale == 1:
        for col in ['keygen_time','enc_time','dec_time']:
            df[col] = df[col].apply(lambda v: '' if pd.isna(v) else str(int(v)))
    else:
        for col in ['keygen_time','enc_time','dec_time']:
            df[col] = df[col].apply(lambda v: format_sig(v/t_scale))
    for col, scale in [('keygen_heap', h_scale), ('enc_heap', h_scale), ('dec_heap', h_scale)]:
        df[col] = df[col].apply(lambda v: format_sig(v/scale))
    for col, scale in [('keygen_stack', s_scale), ('enc_stack', s_scale), ('dec_stack', s_scale)]:
        df[col] = df[col].apply(lambda v: format_sig(v/scale))

    # Header erstellen
    titles = [f'Time ({t_unit})', f'Heap ({h_unit})', f'Stack ({s_unit})']
    headers = ['Model'] + [lbl for t in titles for lbl in ['Key', sub_map['Enc'], sub_map['Dec']]]
    df.columns = headers

    # Header-Gruppen mit Rahmen
    for (title, start, end) in zip(titles, [2,5,8], [4,7,10]):
        ws.merge_cells(start_row=1, start_column=start, end_row=1, end_column=end)
        for col in range(start, end+1):
            cell = ws.cell(row=1, column=col)
            if col == start:
                cell.value     = title
                cell.font      = FONT_BOLD
                cell.alignment = Alignment(horizontal='center', vertical='center')
            cell.border    = border

    # Untertitel (Spaltenüberschriften) ab Spalte 2, Zeile 2
    for idx, hdr in enumerate(headers[1:], start=2):
        c = ws.cell(row=2, column=idx, value=hdr)
        c.font      = FONT_BOLD
        c.alignment = Alignment(horizontal='center', vertical='center')
        c.border    = border

    # Datenzeilen
    for r, row in enumerate(df.itertuples(index=False), start=3):
        for c_idx, val in enumerate(row, start=1):
            cell = ws.cell(row=r, column=c_idx, value=val)
            cell.font      = FONT
            cell.alignment = Alignment(horizontal='right', vertical='center')
            cell.border    = border

    # Spaltenbreiten anpassen
    for ci in range(1, len(headers)+1):
        letter = get_column_letter(ci)
        max_len = 0
        for rr in range(2, ws.max_row+1):
            v = ws.cell(row=rr, column=ci).value
            if v:
                max_len = max(max_len, len(str(v)))
        ws.column_dimensions[letter].width = max_len + 1

    # Zeilenhöhe
    for rr in range(1, ws.max_row+1):
        ws.row_dimensions[rr].height = 12

    wb.save(os.path.join(out_dir, f'{safe}.xlsx'))
    print(f"Einzeldatei für {algo}: {safe}.xlsx")


Einzeldatei für BIG_QUAKE_1: BIG_QUAKE_1.xlsx
Einzeldatei für BIG_QUAKE_3: BIG_QUAKE_3.xlsx
Einzeldatei für BIG_QUAKE_5: BIG_QUAKE_5.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode2_AES: CRYSTALS-DILITHIUM_mode2_AES.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode2_SHA: CRYSTALS-DILITHIUM_mode2_SHA.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode3_AES: CRYSTALS-DILITHIUM_mode3_AES.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode3_SHA: CRYSTALS-DILITHIUM_mode3_SHA.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode5_AES: CRYSTALS-DILITHIUM_mode5_AES.xlsx
Einzeldatei für CRYSTALS-DILITHIUM_mode5_SHA: CRYSTALS-DILITHIUM_mode5_SHA.xlsx
Einzeldatei für Ding_LWE: Ding_LWE.xlsx
Einzeldatei für EMBLEM-EMBLEM: EMBLEM-EMBLEM.xlsx
Einzeldatei für EMBLEM-R_EMBLEM: EMBLEM-R_EMBLEM.xlsx
Einzeldatei für Falcon_1024_fpu: Falcon_1024_fpu.xlsx
Einzeldatei für Falcon_512_fpu: Falcon_512_fpu.xlsx
Einzeldatei für FrodoKEM-1344_AES: FrodoKEM-1344_AES.xlsx
Einzeldatei für FrodoKEM-1344_SHAKE: FrodoKEM-1344_SHAKE.xlsx
Einzeldatei 